# Podatkovno rudarjenje 01 - evidenca nesreč na smučiščih

#### avtor: Urban Papler
#### datum: 30. 3. 2020

V tem notebooku bomo iz evidence nesreč na smučiščih izluščili nekaj zanimivih povezav in jih prikazali.

### Korak 0: branje in priprava podatkov

Preberemo podatke iz datoteke ter jih obdelamo tako, da bo delo z njimi čim lažje.

In [ ]:
# branje podatkov
